# Wilcoxon Signed Rank Test
*By P. Stikker*<br>
https://PeterStatistics.com<br>
https://www.youtube.com/stikpet<br>

## Introduction

To test if scores on two paired ordinal variables are evenly distributed there are two tests often mentioned that can be used. Either a two-sample sign test, or a Wilcoxon signed rank test (Wilcoxon, 1945). In both tests the difference between the two variables for each case (respondent) is calculated first. The two-sample sign test then 'simply' checks if the number of positive differences is the same as the number of negative differences (or at least could be in the population). This test ignores the size of the difference, and this is something the Wilcoxon signed rank test does take into consideration to a certain extend. As the name implies it uses ranks to determine if the sum of the ranks is significantly different between the sum of the ranks of the positive differences and of the ranks of the negative differences. I'll use this test for the example.

Note that the Wilcoxon test actually removes any ties, i.e. if the score on each variable is the same for a case, it will not be used. Pratt (1959) proposed an alternative method, that does still use these tied scores in the ranking, but it is a lot less known. Another approach might be an partially overlapping samples t-test (Derrick & White, 2018), but although this test might actually be the best to use, this would require to make some more assumptions about the data and is not well-known (yet).

## Example

To show an example, I'll load some data as a pandas dataframe. So I'll need the '<a href="https://pandas.pydata.org">pandas</a>' library:

In [1]:
#!pip install pandas
import pandas as pd

And then load the example data using the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html">'read_csv'</a>. 

In [2]:
myDf = pd.read_csv('wil.csv')
myDf.head()

,DC-diger,dc-biz
0,95.22,96.27
1,95.80,96.27
2,80.80,96.27
3,94.80,95.27
4,95.80,96.27


One simple method to perform a Wilcoxon test is to use the '<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wilcoxon.html">wilcoxon</a>' function from the scipy.stats package:

In [3]:
# !pip install scipy
from scipy.stats import wilcoxon

Its important to remove any missing values from our data, so first create a new dataframe without any missing values. This can easily be done using Pandas '<a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html">dropna</a>':

In [4]:
newDf = myDf[['DC-diger', 'dc-biz']].dropna()

We can then perform the test simply:

In [5]:
wilcoxon(newDf['DC-diger'], newDf['dc-biz'], correction=False)

WilcoxonResult(statistic=0.0, pvalue=0.0078125)

This means that there is less than .001 (0.1%) chance to get an absolute a statistic (W) of 85 or even more in a sample if there wouldn't be any difference in mean ranks in the population. 

This chance is so low that most likely there is a difference (usually below .05 is the threshold), which indicates that there is a significant difference between the two variables.

The *correction=False* avoids that scipy uses a continuity correction. If you do want to use this, you can set it to true:

In [6]:
wilcoxon(newDf['DC-diger'], newDf['dc-biz'], correction=True)

WilcoxonResult(statistic=0.0, pvalue=0.0078125)

Scipy can also perform the 'Pratt' version:

In [7]:
wilcoxon(newDf['DC-diger'], newDf['dc-biz'], zero_method='pratt')

WilcoxonResult(statistic=0.0, pvalue=0.0078125)

Another parameter is the *mode*. By default this is set to *auto* which will use an exact Wilcoxon distribution if the number of pairs is 25 or less, and there are no ties, otherwise it will use the approximation with the normal distribution. We can also use mode='exact', but only if the number of pairs is 25 or less (so not in this example).

There are some other packages that can also perform this test, but most of them have less options:

We can also use the '<a href="https://researchpy.readthedocs.io/en/latest/ttest_documentation.html">ttest</a>' function from researchpy to perform the test:

In [8]:
# !pip install PyNonpar
from researchpy import ttest

In [10]:
rpRes = ttest(newDf['DC-diger'], newDf['dc-biz'], equal_variances=False, paired=True)
rpRes

,Wilcoxon signed-rank test,results
0,Mean for DC-diger =,93.17750
1,Mean for dc-biz =,96.09875
2,T value =,0.00000
3,Z value =,-2.52050
4,Two sided p value =,0.00780
5,r =,-0.63010


The z-value reported here is not adjusted for ties. Its based on the formula:

\begin{equation*}
Z = \frac{W - \mu}{\sqrt{\sigma^2}}
\end{equation*}

In this formula $W$ is the W-statistic (the T in the output above). For $\mu$ the following formula is used:

\begin{equation*}
\mu = \frac{n_r\times\left(n_r+1\right)}{4}
\end{equation*}

Where $n_r$ is the number of ranks (which is the number of pairs that are not equal), and is $\sigma^2$ defined as:

\begin{equation*}
\sigma^2 = \frac{n_r\times\left(n_r+1\right)\times\left(2\times n_r+1\right)}{6}
\end{equation*}

We can also use the '<a href="https://pingouin-stats.org/generated/pingouin.wilcoxon.html">wilcoxon</a>' function from pingouin:

In [11]:
from pingouin import wilcoxon as pgWilc

In [13]:
pgWilc(newDf['DC-diger'], newDf['dc-biz'])

,W-val,tail,p-val,RBC,CLES
Wilcoxon,0.0,two-sided,0.007812,-1.0,0.15625


Note that pingouin always applies the correction.

For those interested, in the appendix I'll go over the formulas involved and avoid using libraries as much as possible (only for the normal distribution to get the eventuel p-value).

## References

Derrick, B., & White, P. (2018). Methods for comparing the responses from a Likert question, with paired observations and independent observations in each of two samples. *International Journal of Mathematics and Statistics, 19*(3), 84–93.

Pratt, J. W. (1959). Remarks on Zeros and Ties in the Wilcoxon Signed Rank Procedures. *Journal of the American Statistical Association, 54*(287), 655–667. doi:10.1080/01621459.1959.10501526

Wilcoxon, F. (1945). Individual comparisons by ranking methods. *Biometrics Bulletin, 1*(6), 80. doi:10.2307/3001968


## Appendix: The Hard Way

First we convert our pandas series to a Python native format: a list

In [14]:
list1 = list(newDf['DC-diger'])
list2 = list(newDf['dc-biz'])

Now to create a list of tuples, excluding any situation where the two values are equal:

In [15]:
X = []
for i in range(len(list1)):
    if list1[i]!=list2[i]:
        X = X + [(list1[i], list2[i])]

print(X)

[(95.22, 96.27), (95.8, 96.27), (80.8, 96.27), (94.8, 95.27), (95.8, 96.27), (96.0, 98.0), (94.0, 95.22), (93.0, 95.22)]


We can use Python native's len function to determine the number of pairs we now have (nr):

In [16]:
nr = len(X)
nr

8

Now for each tuple determine the absolute difference, and also keep track of which one was larger.

In formula notation:

\begin{equation*}
D_i = |X_{i_x} - X_{i_y}|
\end{equation*}

In this formula $X_{i_x}$ is the x-value of the i-the pair, and $X_{i_y}$ the y value.

For the sign we can use:
\begin{equation*}
sign = \left\{\begin{matrix}
-1 & \text{if } X_{i_x}<X_{i_y}\\ 
1 & \text{if } X_{i_x}>X_{i_y} 
\end{matrix}\right.
\end{equation*}

In [17]:
D = []
sg = []
for i in range(nr):
    D = D + [abs(X[i][0]-X[i][1])]
    if X[i][0]>X[i][1]:
        sg = sg + [1]
    else:
        sg = sg + [-1]
    
print(D)
print(sg)

[1.0499999999999972, 0.46999999999999886, 15.469999999999999, 0.46999999999999886, 0.46999999999999886, 2.0, 1.2199999999999989, 2.219999999999999]
[-1, -1, -1, -1, -1, -1, -1, -1]


We need to rank these differences. I'll re-use my rank function made for the Spearman rho (see separate documentation for some more details). The adjustment made here, is that it will also return a dictionary with the frequency of each unique rank, which will come in handy later.

In [18]:
def rankList2(aList):
    sortList = aList.copy()
    sortList.sort()
    
    latestRank = 1
    rankDict = {}
    rankFreq = {}
    freqScore = 1
    for i in range(1, len(sortList)):

        if sortList[i] == sortList[i-1]:
            freqScore = freqScore + 1

        if sortList[i] != sortList[i-1]:
            rank = (2*latestRank + freqScore - 1) / 2

            rankDict[sortList[i-1]] = rank
            rankFreq[rank] = freqScore
            latestRank = latestRank + freqScore
            freqScore = 1

    # the last case
    rankDict[sortList[len(sortList)-1]] = (2*latestRank + freqScore - 1) / 2
    rankFreq[(2*latestRank + freqScore - 1) / 2] = freqScore
    
    # replace list scores with rank scores
    allRanks = []
    for i in aList:
        allRanks.append(rankDict[i])
    
    return rankFreq, allRanks

In [19]:
rFreq, R = rankList2(D)
print(R)
print(rFreq)

[4.0, 2.0, 8.0, 2.0, 2.0, 6.0, 5.0, 7.0]
{2.0: 3, 4.0: 1, 5.0: 1, 6.0: 1, 7.0: 1, 8.0: 1}


We can now determine two W-values. One for all the positive ranks, and one for all the negative:

\begin{equation*}
W_+ = \sum_{i \text{ if } sign(i)>0}R_i
\end{equation*}

and

\begin{equation*}
W_- = \sum_{i \text{ if } sign(i)<0}R_i
\end{equation*}

In [20]:
Wplus = 0
Wmin = 0

for i in range(nr):
    if sg[i]>0:
        Wplus = Wplus + R[i]
    else:
        Wmin = Wmin + R[i]
        
Wplus, Wmin

(0, 36.0)

Now we can calculate:

\begin{equation*}
\mu = \frac{n_r\times\left(n_r+1\right)}{4}
\end{equation*}


In [21]:
m = nr * (nr + 1) / 4
m

18.0

Also:

\begin{equation*}
\sigma^2 = \frac{n_r\times\left(n_r+1\right)\times\left(2\times n_r+1\right)}{24}
\end{equation*}

In [22]:
var = nr * (nr + 1) * (2*nr + 1) / 24
var

51.0

To determine the correction for ties, we calculate first:

\begin{equation*}
T = \sum \left( t_{i}^3 - t_i \right)
\end{equation*}

In [23]:
T = 0
for key in rFreq:
    T = T + rFreq[key]**3 - rFreq[key]
    
T

24

The adjusted variance is then given by:

\begin{equation*}
\sigma_{adj}^2 = \sigma^2 - \frac{T}{48}
\end{equation*}


In [24]:
varAdj = var - T/48
varAdj

50.5

The standard error can then be defined as:

\begin{equation*}
SE_{adj} = \sqrt{\sigma_{adj}^2}
\end{equation*}

In [25]:
SEadj = varAdj**0.5
SEadj

7.106335201775948

And finally the z-value as:

\begin{equation*}
Z = \frac{W_+}{SE_{adj}}
\end{equation*}

or
\begin{equation*}
Z = \frac{W_-}{SE_{adj}}
\end{equation*}

The difference between these two will only be in a negative sign in front of one, and not the other.

In [26]:
Z1 = (Wplus - m) / SEadj
Z1

-2.5329511610290507

In [27]:
Z2 = (Wmin - m) / SEadj
Z2

2.5329511610290507

We can use scipy.stats now to convert these z-values to a p-value using the standard normal distribution:

In [28]:
from scipy.stats import norm

In [29]:
2*(1-norm.cdf(abs(Z2)))

0.011310671074428624

In [30]:
2*(1-norm.cdf(abs(Z2)))

0.011310671074428624